In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

--2022-11-14 12:00:37--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.45.112, 142.250.73.240, 172.253.62.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.45.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip.1’

horse-or-human.zip. 100%[===================>] 142.65M   232MB/s    in 0.6s    

2022-11-14 12:00:37 (232 MB/s) - ‘horse-or-human.zip.1’ saved [149574867/149574867]



In [ ]:
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

--2022-11-14 12:00:39--  https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.45.112, 142.250.73.240, 172.253.62.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.45.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]
Saving to: ‘validation-horse-or-human.zip.1’

validation-horse-or 100%[===================>]  10.95M  --.-KB/s    in 0.1s    

2022-11-14 12:00:39 (115 MB/s) - ‘validation-horse-or-human.zip.1’ saved [11480187/11480187]



In [ ]:
import zipfile

local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')


In [ ]:
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

In [ ]:
import os
image_dir=os.path.join('./horse-or-human')
val_dir=os.path.join('./validation-horse-or-human')
trainhorse= os.path.join('./horse-or-human/horses')
trainhuman=os.path.join('./horse-or-human/humans')
valhorse=os.path.join('./validation-horse-or-human/horses')
valhuman=os.path.join('./validation-horse-or-human/humans')

In [ ]:
train_horse_names = os.listdir(trainhorse)
print(train_horse_names[:10])

train_human_names = os.listdir(trainhuman)
print(train_human_names[:10])

validation_human_names=os.listdir(valhuman)
validation_horse_names=os.listdir(valhorse)



['horse11-2.png', 'horse05-4.png', 'horse29-8.png', 'horse42-5.png', 'horse30-0.png', 'horse09-6.png', 'horse28-8.png', 'horse23-8.png', 'horse44-0.png', 'horse26-5.png']
['human01-01.png', 'human11-12.png', 'human05-05.png', 'human09-29.png', 'human16-27.png', 'human09-04.png', 'human05-18.png', 'human03-05.png', 'human09-18.png', 'human13-12.png']


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [ ]:
train_data=ImageDataGenerator(rescale=1./255)
train_gen= train_data.flow_from_directory(image_dir, target_size=(300,300), batch_size=128, class_mode='binary')

val_data=ImageDataGenerator(rescale=1./255)
val_gen= train_data.flow_from_directory(val_dir, target_size=(300,300), batch_size=32, class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
model=tf.keras.Sequential([tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)), tf.keras.layers.MaxPooling2D(2,2)  , tf.keras.layers.Conv2D(32, (3,3), activation='relu'), tf.keras.layers.MaxPooling2D(2,2),  tf.keras.layers.Conv2D(32, (3,3), activation='relu'), tf.keras.layers.MaxPooling2D(2,2) ,  tf.keras.layers.Flatten(), tf.keras.layers.Dense(128, activation='relu'), tf.keras.layers.Dense(1, activation='sigmoid')])

In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get('accuracy')>98.5):
      print("stopp")
      self.model.stop_training=True

callbacks=myCallback()

In [ ]:
model.fit(
      train_gen,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = val_gen,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 17s 917ms/step - loss: 3.0348 - accuracy: 0.5879 - val_loss: 0.7580 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 7s 982ms/step - loss: 0.6900 - accuracy: 0.7275 - val_loss: 0.4488 - val_accuracy: 0.7227
Epoch 3/15
8/8 [==============================] - 8s 934ms/step - loss: 0.4317 - accuracy: 0.8145 - val_loss: 1.2717 - val_accuracy: 0.5938
Epoch 4/15
8/8 [==============================] - 8s 931ms/step - loss: 0.2600 - accuracy: 0.8848 - val_loss: 1.2153 - val_accuracy: 0.7617
Epoch 5/15
8/8 [==============================] - 7s 877ms/step - loss: 0.1476 - accuracy: 0.9388 - val_loss: 2.2277 - val_accuracy: 0.6836
Epoch 6/15
8/8 [==============================] - 7s 898ms/step - loss: 0.9097 - accuracy: 0.8087 - val_loss: 1.3774 - val_accuracy: 0.7109
Epoch 7/15
8/8 [==============================] - 7s 892ms/step - loss: 0.1074 - accuracy: 0.9622 - val_loss: 2.2816 - val_accuracy: 0.6875
Epoch 8/15
8/8 [===

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(300, 300))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
    
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving download.jpg to download (1).jpg
1/1 [==============================] - 0s 16ms/step
[6.8407585e-20]
download.jpg is a horse
